# Установка

In [2]:
! pip install ortools
! pip install matplotlib seaborn pandas numpy

  Using cached matplotlib-3.10.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached contourpy-1.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.4-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (106 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached pillow-11.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached contourpy-1.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinu

# Импорты

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from ortools.sat.python import cp_model


# Анализ

In [ ]:
# Прочитаем данные
bulls = pd.read_csv('../data/bulls.csv')
print(bulls.head())
bulls.info()
# Будем считать, что descendants_count - кол-во возможных детей

              id  descendants_count     ebv
0  FR00000000479                476   690.7
1  NL00000001842                325  -597.6
2  US00000003013                436  1528.6
3  US00000001653               1097  1193.2
4  DE00000001742                496  1510.6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 39 non-null     object 
 1   descendants_count  39 non-null     int64  
 2   ebv                38 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.0+ KB


In [6]:
# Прочитаем данные
cows = pd.read_csv('../data/cows.csv')
print(cows.head())
cows.info()

              id    ebv
0  RU00000090385  266.5
1  DE00000090396    NaN
2  NL00000090422 -582.8
3  GB00000090437  443.8
4  DE00000090443  173.3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17177 entries, 0 to 17176
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      17177 non-null  object 
 1   ebv     15328 non-null  float64
dtypes: float64(1), object(1)
memory usage: 268.5+ KB


In [7]:
# Прочитаем данные
pedigree = pd.read_csv('../data/pedigree.csv')
print(pedigree.head())
pedigree.info()

              id      mother_id      father_id
0  GB00000090350  GB00000070596  FR00000051087
1  DE00000090351  GB00000085021  GB00000051158
2  US00000090352  US00000087323  GB00000091078
3  FR00000090353            NaN            NaN
4  GB00000090358  US00000056066  NL00000050889
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94400 entries, 0 to 94399
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         94400 non-null  object
 1   mother_id  87822 non-null  object
 2   father_id  89562 non-null  object
dtypes: object(3)
memory usage: 2.2+ MB


# Создание матрицы потенциальных назначений

In [ ]:
# Сопоставим каждой особе список предков до 5 колена. Т.к. все, что идет дальше соответствует <5% родства. Для этого будем углубляться по дереву предков

def find_parents(animal_id, cnt=0, curr_result=None):
    if curr_result is None:
        curr_result = []
    
    if cnt >= 5:
        return curr_result
    
    animal_row = pedigree[pedigree['id'] == animal_id]
    
    if animal_row.empty:
        return curr_result
    
    mother_id = animal_row['mother_id'].values[0]
    father_id = animal_row['father_id'].values[0]
    
    # Добавление родителей, если они известны
    if pd.notna(mother_id):
        curr_result.append(mother_id)
        find_parents(mother_id, cnt + 1, curr_result)
    
    if pd.notna(father_id):
        curr_result.append(father_id)
        find_parents(father_id, cnt + 1, curr_result)
    
    return curr_result
find_parents('GB00000090350')


['GB00000070596',
 'NL00000074750',
 'DE00000061546',
 'FR00000078118',
 'US00000090614',
 'DE00000058308',
 'FR00000091137',
 'FR00000088924',
 'NL00000051203',
 'NL00000051115',
 'US00000050818',
 'FR00000050367',
 'FR00000050361',
 'DE00000050293',
 'US00000050750',
 'US00000050615',
 'DE00000050591',
 'DE00000050607',
 'RU00000050704',
 'DE00000050767',
 'RU00000050613',
 'RU00000050704',
 'NL00000050382',
 'RU00000050408',
 'FR00000050407',
 'FR00000050332',
 'DE00000050429',
 'GB00000050433',
 'US00000050417',
 'FR00000051087',
 'NL00000050932',
 'NL00000051042',
 'NL00000050940',
 'RU00000050608',
 'GB00000050770',
 'FR00000050578',
 'US00000050723',
 'NL00000050481',
 'DE00000050666',
 'US00000050639',
 'FR00000050558',
 'FR00000050808',
 'GB00000050577',
 'FR00000050545',
 'NL00000050481',
 'RU00000050593',
 'RU00000050778',
 'US00000050641',
 'NL00000050481',
 'FR00000050401',
 'RU00000050664',
 'FR00000050642',
 'NL00000050547']

In [11]:
# Сделаем это для всех особей
bulls['valid_ancestors'] = bulls['id'].apply(find_parents)
cows['valid_ancestors'] = cows['id'].apply(find_parents)
print(bulls.head())
print(cows.head())

              id  descendants_count     ebv  \
0  FR00000000479                476   690.7   
1  NL00000001842                325  -597.6   
2  US00000003013                436  1528.6   
3  US00000001653               1097  1193.2   
4  DE00000001742                496  1510.6   

                                     valid_ancestors  
0  [US00000000802, FR00000000028, GB00000000783, ...  
1  [DE00000001681, DE00000002197, NL00000002220, ...  
2  [US00000002320, DE00000002323, DE00000002324, ...  
3  [FR00000001777, DE00000001940, RU00000002216, ...  
4  [RU00000002032, RU00000001712, RU00000001939, ...  
              id    ebv                                    valid_ancestors
0  RU00000090385  266.5  [US00000093522, GB00000053019, RU00000054636, ...
1  DE00000090396    NaN  [RU00000067677, NL00000068009, DE00000060726, ...
2  NL00000090422 -582.8  [NL00000053949, FR00000055560, US00000064902, ...
3  GB00000090437  443.8  [DE00000075045, RU00000056086, RU00000078370, ...
4  DE0000009

# Алгоритмы:

Здесь постараемся рассмотреть несколько вариантов:

1) Модель линейной оптимизации (ortools, Задача назначений)

2) Жадный алгоритм

3) Генетический алгоритм


И сравним полученные значения EBV

## Линейная оптимизация со следующими ограничениями:

### Критерии и ограничения, которые нужно учитывать

1.​ Ожидаемый EBV потомка

    ○​ Рассчитывается как среднее значение между EBV быка и коровы.
    
    ○​ Необходимо максимизировать среднее значение по всему закреплению.

2.​ Разнообразие потомства
    
    ○​ При прочих равных предпочтение даётся таким парам, где разброс EBV
потомков больше.

3.​ Генетическое разнообразие

    ○​ Уровень родства в паре не должен превышать 5%.

4.​ Ограничение по использованию быков

    ○​ Один бык не может осеменить более 10% коров.


### Функция оптимизации:

sum(EBVi), i = 1...n


In [2]:
cows = pd.read_csv('../data/ppcd_cows.csv')
bulls = pd.read_csv('../data/ppcd_bulls.csv')

In [3]:
cows.dropna(inplace=True)
bulls.dropna(inplace=True) # удаляем т.к. допускаем, чтоо не повлияют на исход работы алгоритма
cows['ebv'] = cows['ebv'].apply(lambda x: int(x * 100))
bulls['ebv'] = bulls['ebv'].apply(lambda x: int(x * 100)) # для решения в целых числах
cows.head()

,Unnamed: 0,id,ebv,valid_ancestors
0,0,RU00000090385,26650,"['US00000093522', 'GB00000053019', 'RU00000054..."
1,1,NL00000090422,-58279,"['NL00000053949', 'FR00000055560', 'US00000064..."
2,2,GB00000090437,44380,"['DE00000075045', 'RU00000056086', 'RU00000078..."
3,3,DE00000090443,17330,"['US00000093024', 'NL00000055094', 'FR00000086..."
4,4,DE00000090473,1200,"['NL00000073771', 'US00000080682', 'DE00000054..."


In [4]:
import math
from ortools.sat.python import cp_model

# Предобработка данных: удаление коров с NaN в 'ebv'
cows = cows.dropna(subset=['ebv']).reset_index(drop=True)

# Расчёт квоты: 10% от количества коров с округлением вверх
quota = math.ceil(cows.shape[0] * 0.1)

# Функция для расчёта родства
def calculate_kinship(bull_ancestors, cow_ancestors):
    set_bull = set(bull_ancestors)
    set_cow = set(cow_ancestors)
    return int(len(set_bull & set_cow) > 0)

num_cows = len(cows)
num_bulls = len(bulls)

# Создание модели
model = cp_model.CpModel()

# Переменные: x[cow_idx][bull_idx] = 1, если корова cow_idx закреплена за быком bull_idx
x = {}
for cow_idx in range(num_cows):
    for bull_idx in range(num_bulls):
        x[cow_idx, bull_idx] = model.NewBoolVar(f'x_{cow_idx}_{bull_idx}')

# Ограничение 1: каждая корова закреплена за одним быком
for cow_idx in range(num_cows):
    model.Add(sum(x[cow_idx, bull_idx] for bull_idx in range(num_bulls)) == 1)

# Ограничение 2: один бык не может обслужить более quota коров
for bull_idx in range(num_bulls):
    model.Add(sum(x[cow_idx, bull_idx] for cow_idx in range(num_cows)) <= quota)

# Целевая функция: максимизация суммы EBV с учётом штрафа за родство
objective_terms = []
for cow_idx in range(num_cows):
    cow_row = cows.iloc[cow_idx]
    cow_ancestors = cow_row['valid_ancestors']
    cow_ebv = cow_row['ebv']
    
    for bull_idx in range(num_bulls):
        bull_row = bulls.iloc[bull_idx]
        bull_ancestors = bull_row['valid_ancestors']
        bull_ebv = bull_row['ebv']
        
        # Проверка родства
        kinship = calculate_kinship(bull_ancestors, cow_ancestors)
        weight = 1.0 if kinship == 0 else 0.01  # Штраф за родство
        
        # Добавление в целевую функцию
        objective_terms.append(((bull_ebv + cow_ebv) / 2) * weight * x[cow_idx, bull_idx])

model.Maximize(sum(objective_terms))

# Решение
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 60

status = solver.Solve(model)

# Вывод результатов
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE, 1, 2):
    print("Оптимальное решение найдено.")
    print(f" Значение целевой функции: {solver.ObjectiveValue()}")

    assignments = []
    for cow_idx in range(num_cows):
        for bull_idx in range(num_bulls):
            if solver.Value(x[cow_idx, bull_idx]) == 1:
                cow_id = cows.iloc[cow_idx]['id']
                bull_id = bulls.iloc[bull_idx]['id']
                assignments.append((cow_id, bull_id))
    print("Назначения (корова -> бык):", assignments[:5])  
else:
    print("Решение не найдено.")
    print('status:', status)

: 